In [ ]:
!nvidia-smi

In [ ]:
import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Util function for loading point clouds|
import numpy as np

# Data structures and functions for rendering
from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras, 
    FoVPerspectiveCameras,
    PointsRasterizationSettings,
    RasterizationSettings,
    PointsRenderer,
    PulsarPointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
    NormWeightedCompositor)

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

from torchvision.models.efficientnet_v2_m import EfficientNet, EfficientNet_V2_M_Weights

In [ ]:
EfficientNet(weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1)
transform_img = EfficientNet_V2_M_Weights.IMAGENET1K_V1.transforms

num_points = 10
size = 10
verts = (torch.rand(1, num_points, 3).to(device) * size) - (size/2)
rgb = torch.rand(1, num_points, 3).to(device)
point_cloud = Pointclouds(points=verts, features=rgb)

R, T = look_at_view_transform(dist=15, elev=10, azim=0)
# cameras = FoVOrthographicCameras(device=device, R=R, T=T, znear=0.01)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T, znear=0.01, fov=60)

raster_settings = PointsRasterizationSettings(image_size=1024, radius = 0.03, points_per_pixel = 1)

# Create a points renderer by compositing points using an alpha compositor (nearer points
# are weighted more heavily). See [1] for an explanation.
rasterizer = PointsRasterizer(cameras=cameras, raster_settings=raster_settings)
        
renderer = PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor()
)

images = renderer(point_cloud)
plt.figure(figsize=(10, 10))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.axis("off")

In [ ]:
renderer = PulsarPointsRenderer(
    rasterizer=PointsRasterizer(cameras=cameras, raster_settings=raster_settings),
    n_channels=4
).to(device)

images = renderer(point_cloud, gamma=(1e-4,),
                  bg_col=torch.tensor([0.0, 1.0, 0.0, 1.0], dtype=torch.float32, device=device))
plt.figure(figsize=(10, 10))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.axis("off")